In [13]:
import pandas as pd
pd.set_option('display.max_columns', None)
import sys
sys.path.append('../../')

titanic_df = pd.read_csv('titanic_data/data.csv')
titanic_df['Survived'] = titanic_df['Survived'].astype(str)

In [14]:
import tabularmagic as tm
analyzer = tm.Analyzer(titanic_df, test_size=0.2, name='titanic')
analyzer

INFO: Analyzer initialized. Shapes of train, test DataFrames: (712, 12), (179, 12). 


titanic 
----------------------------------------------------------------------------------------
Train shape: (712, 12)                      Test shape: (179, 12)                       
----------------------------------------------------------------------------------------
Categorical variables:
  'Survived', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked' 
                                                                                        
Numeric variables:
  'PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare' 

In [18]:
report = analyzer.classify(
    models=[
        tm.ml.LinearC('no_penalty'),
        tm.ml.LinearC('l1'),
        tm.ml.LinearC('l2'),
        tm.ml.TreeEnsembleC('random_forest', n_trials=10)
    ],
    target='Survived',
    predictors=[
        'Pclass',
        'Sex',
        'Fare',
        'Age'
    ]
)

INFO: Evaluating model LinearC(no_penalty). 
WARN: Train data: dropped 140 rows with missing values out of a total of 712 rows. 
PROG: Fitting LinearC(no_penalty). Search method: GridSearchCV (1 fits per fold, 5 total 
      fits). 
PROG: Optimal threshold set for LinearC(no_penalty) via F1 score. 
WARN: Test data: dropped 37 rows with missing values out of a total of 179 rows. 
INFO: Successfully evaluated model LinearC(no_penalty). 
INFO: Evaluating model LinearC(l1). 
WARN: Train data: dropped 140 rows with missing values out of a total of 712 rows. 
PROG: Fitting LinearC(l1). Search method: OptunaSearchCV (100 trials). 
PROG: Optimal threshold set for LinearC(l1) via F1 score. 
WARN: Test data: dropped 37 rows with missing values out of a total of 179 rows. 
INFO: Successfully evaluated model LinearC(l1). 
INFO: Evaluating model LinearC(l2). 
WARN: Train data: dropped 140 rows with missing values out of a total of 712 rows. 
PROG: Fitting LinearC(l2). Search method: OptunaSearchCV 

In [19]:
report.metrics()

,LinearC(no_penalty),LinearC(l1),LinearC(l2),TreeEnsembleC(random_forest)
Statistic,,,,
accuracy,0.823944,0.816901,0.838028,0.823944
f1,0.80916,0.80303,0.809917,0.774775
precision,0.726027,0.716216,0.777778,0.811321
recall,0.913793,0.913793,0.844828,0.741379
roc_auc,0.891626,0.889368,0.899425,0.887726
n_obs,142,142,142,142


In [21]:
report.feature_importance('LinearC(no_penalty)')

,Abs Coefs
Feature,
male_yn(Sex),2.477569
Pclass,1.085313
Fare,0.001015
Age,0.027608


In [22]:
glm_report = analyzer.glm(
    'binomial',
    target='Survived',
    predictors=[
        'Pclass',
        'Sex',
        'Fare',
        'Age'
    ]
)

WARN: Train data: dropped 140 rows with missing values out of a total of 712 rows. 


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [23]:
glm_report.statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                  572
Model:                            GLM   Df Residuals:                      567
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -267.21
Date:                Sun, 04 Aug 2024   Deviance:                       534.42
Time:                        06:10:40   Pearson chi2:                     595.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3404
Covariance Type:                  HC3                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.3005      0.602      7.147      0.000       3.121       5.480
male_yn(Sex)    -2.4775      0.220    -11.276      0.000      -2.908      -2.047
Pclass          -1.0853      0.174     -6.234      0.000      -1.427      -0.744
Fare             0.0010      0.002      0.502      0.616      -0.003       0.005
Age             -0.0276      0.009     -3.182      0.001      -0.045      -0.011
================================================================================
"""